In [1]:
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


In [2]:
from gensim.models import KeyedVectors
from tqdm import tqdm
import pandas as pd
import re
import torch

In [3]:
PATH = "/content/drive/MyDrive/dataset/ch8/"
device = "cuda" if torch.cuda.is_available() else "cpu"
print("Using {} device".format(device))

Using cuda device


In [ ]:
# 70
# CATEGORYをencodeする
# b = business, t = science and technology, e = entertainment, m = health
def EncoderNN(sign):
    if sign == "b":
        code = 0
    elif sign == "t":
        code = 1
    elif sign == "e":
        code = 2
    elif sign == "m":
        code = 3
    else:
        print("Error")
    return code

# テキストをベクトルに変換する
def Text2Vec(text):
    lines = text.split(" ")
    # sapceで分けて

    vec_sum = 0
    length = 0

    for line in lines:
      # 単語を一つずつ読み込む
        try:
            temp = model.get_vector(line)
            # 単語のベクトルをもらう
            vec_sum += temp
            # ベクトルを合計する
            length += 1
        except:
            pass
            # 単語が存在しない場合は無視する

    # 平均ベクトルを計算する
    return vec_sum/length

def TorchData(data):
    df = pd.read_table(PATH + "{}.txt".format(data))
    # dataを読み込む

    sign_regrex = re.compile('[!"#$%&\'()*+,-./:;<=>?@[\\]^_`|＄＃＠£â€™]')
    f_regrex = lambda x:sign_regrex.sub("", x)
    df["TITLE"] = df["TITLE"].map(f_regrex)
    # TITLE内の記号など削除する


    X_torch = torch.tensor(df["TITLE"].apply(Text2Vec)).to(device)
    torch.save(X_torch, PATH + "X_{}.pt".format(data))
    df["CATEGORY"] = df["CATEGORY"].map(EncoderNN)
    Y_torch = torch.tensor(df["CATEGORY"]).to(device)
    torch.save(Y_torch, PATH + "Y_{}.pt".format(data))
    # TITLEとCATEGORYの単語のベクトルを計算して、保存する

model = KeyedVectors.load_word2vec_format(PATH + "GoogleNews-vectors-negative300.bin", binary=True)
TorchData("train")
TorchData("test")
TorchData("valid")

In [4]:
# 71
import torch.nn as nn
import torch

class NeuralNetwork(nn.Module):
    def __init__(self, input_feature, output):
      super(NeuralNetwork, self).__init__()
      self.fc1 = nn.Linear(input_feature, output, bias=False)
      # このレイヤーはinput_featureからoutputへ線形変換をほどこする
      # 300—>4

      self.fc2 = nn.Softmax(dim=1)
      # Softmax関数をほどこする

    def forward(self, x):
      x = self.fc1(x)
      x = self.fc2(x)
      return x

X_train = torch.load(PATH + "X_train.pt")
model = NeuralNetwork(300, 4).to(device)
model(X_train)

tensor([[0.2562, 0.2501, 0.2358, 0.2579],
        [0.2399, 0.2478, 0.2511, 0.2613],
        [0.2459, 0.2546, 0.2576, 0.2419],
        ...,
        [0.2531, 0.2581, 0.2384, 0.2505],
        [0.2487, 0.2440, 0.2439, 0.2635],
        [0.2536, 0.2507, 0.2392, 0.2566]], device='cuda:0',
       grad_fn=<SoftmaxBackward0>)

In [5]:
# 72
loss_function = nn.CrossEntropyLoss(reduction="mean")

Y_train = torch.load(PATH + "Y_train.pt")
# dataを読み込む

Y_pred = model(X_train).to(device)
# X_trainで予測したY_pred

loss = loss_function(Y_pred, Y_train).to(device)


model.zero_grad()
# モデルパラメータの勾配をリセットする
loss.backward()
# バックプロパゲーションを実行します。

print("loss", loss.item())
print("勾配", model.fc1.weight.grad)


loss 1.389650583267212
勾配 tensor([[ 0.0005, -0.0011,  0.0008,  ..., -0.0020, -0.0029,  0.0033],
        [ 0.0007,  0.0009, -0.0016,  ..., -0.0002,  0.0021, -0.0006],
        [-0.0027, -0.0011,  0.0028,  ...,  0.0029, -0.0010, -0.0012],
        [ 0.0015,  0.0013, -0.0020,  ..., -0.0007,  0.0017, -0.0015]],
       device='cuda:0')


In [6]:
# 73
from torch.utils.data import TensorDataset, DataLoader
import torch
import torch.nn as nn
import torch.optim as optim

training_data = TensorDataset(X_train, Y_train)
train_dataloader = DataLoader(training_data,batch_size=128, shuffle=True)

optimizer = torch.optim.SGD(model.parameters(), lr=1e-3)

def train(dataloader, model, loss_fn, optimizer):
    size = len(dataloader.dataset)

    for batch, (X, y) in enumerate(dataloader):
        X, y = X.to(device), y.to(device)

        # 予測とlossの計算
        pred = model(X)
        loss = loss_fn(pred, y)

        # バックプロパゲーション
        optimizer.zero_grad()
        # モデルパラメータの勾配をリセットする
        loss.backward()
        # バックプロパゲーションを実行する
        optimizer.step()
        # パラメータの勾配を使用してパラメータの値を調整する

        # 10バッチごとに、現在のlossとトレーニングの進捗状況をprintして
        if batch % 10 == 0:
            loss, current = loss.item(), batch * len(X)
            print(f"loss: {loss:>7f}  [{current:>5d}/{size:>5d}]")

epochs = 10
for t in range(epochs):
    print(f"Epoch {t+1}\n-------------------------------")
    train(train_dataloader, model, loss_function, optimizer)
    print("-------------------------------\n")

torch.save(model.state_dict(), PATH + "SigleLayer.pth")
#モデルの保存
print("Done!")

Epoch 1
-------------------------------
loss: 1.390280  [    0/10672]
loss: 1.388236  [ 1280/10672]
loss: 1.389010  [ 2560/10672]
loss: 1.389044  [ 3840/10672]
loss: 1.389019  [ 5120/10672]
loss: 1.389305  [ 6400/10672]
loss: 1.390626  [ 7680/10672]
loss: 1.388669  [ 8960/10672]
loss: 1.389453  [10240/10672]
-------------------------------

Epoch 2
-------------------------------
loss: 1.388664  [    0/10672]
loss: 1.389804  [ 1280/10672]
loss: 1.390744  [ 2560/10672]
loss: 1.391934  [ 3840/10672]
loss: 1.389822  [ 5120/10672]
loss: 1.389946  [ 6400/10672]
loss: 1.389788  [ 7680/10672]
loss: 1.387210  [ 8960/10672]
loss: 1.388780  [10240/10672]
-------------------------------

Epoch 3
-------------------------------
loss: 1.387547  [    0/10672]
loss: 1.387555  [ 1280/10672]
loss: 1.387224  [ 2560/10672]
loss: 1.388970  [ 3840/10672]
loss: 1.388766  [ 5120/10672]
loss: 1.387269  [ 6400/10672]
loss: 1.387501  [ 7680/10672]
loss: 1.388736  [ 8960/10672]
loss: 1.389049  [10240/10672]
----

In [16]:
# 74
model.load_state_dict(torch.load(PATH + "SigleLayer.pth"))

X_test = torch.load(PATH + "X_test.pt").to(device)
Y_test = torch.load(PATH + "Y_test.pt").to(device)


X_train = torch.load(PATH + "X_train.pt").to(device)
Y_train = torch.load(PATH + "Y_train.pt").to(device)

# 訓練データ
Y_pred = model(X_train).to(device)

# 予測したデータから最大の要素をもらう
predicted_classes = torch.max(Y_pred.data, dim=1).indices

# 予測したものとY_trainを比較して
correct_predictions = torch.eq(predicted_classes, Y_train)

# 正解率を計算する
accuracy = torch.mean(correct_predictions.float())
print(f"訓練データの正解率: {accuracy.item() * 100:.2f}%")


# 評価データ
Y_pred = model(X_test).to(device)

# 予測したデータから最大の要素をもらう
predicted_classes = torch.max(Y_pred.data, dim=1).indices

# 予測したものとY_testを比較して
predictions = torch.eq(predicted_classes, Y_test)

# 正解率を計算する
accuracy = torch.mean(predictions.float())
print(f"評価データの正解率: {accuracy.item() * 100:.2f}%")

訓練データの正解率: 35.54%
評価データの正解率: 35.46%


In [ ]:
# 75
from torch.utils.data import TensorDataset, DataLoader
from torch.utils.tensorboard import SummaryWriter
import torch
import torch.nn as nn
import torch.optim as optim


def TensorboardWriter(y, pred, epoch, loss, name):


  # 予測したデータから最大の要素をもらう
  predicted_classes = torch.max(pred.data, dim=1).indices

  # 予測したものとY_testを比較して
  predictions = torch.eq(predicted_classes, y)

  # 正解率を計算する
  accuracy = torch.mean(predictions.float())

  # プロットする
  writer.add_scalar("Loss/{}_Loss".format(name), loss, epoch)
  writer.add_scalar("Accuracy/{}_Accuracy".format(name), accuracy, epoch)


writer = SummaryWriter(log_dir="logs")
# プロットのため使用する

X_valid = torch.load(PATH + "X_valid.pt").to(device)
Y_valid = torch.load(PATH + "Y_valid.pt").to(device)

training_data = TensorDataset(X_train, Y_train)
train_dataloader = DataLoader(training_data,batch_size=128, shuffle=True)

test_data = TensorDataset(X_valid, Y_valid)
test_dataloader = DataLoader(test_data,batch_size=128, shuffle=True)

optimizer = torch.optim.SGD(model.parameters(), lr=1e-3)

def train(dataloader, model, loss_fn, optimizer, epochs, name):
    size = len(dataloader.dataset)

    for batch, (X, y) in enumerate(dataloader):
        X, y = X.to(device), y.to(device)

        # 予測とlossの計算
        pred = model(X)
        loss = loss_fn(pred, y)

        # バックプロパゲーション
        optimizer.zero_grad()
        # モデルパラメータの勾配をリセットする
        loss.backward()
        # バックプロパゲーションを実行する
        optimizer.step()
        # パラメータの勾配を使用してパラメータの値を調整する

        # 10バッチごとに、現在のlossとトレーニングの進捗状況をprintして
        # if batch % 10 == 0:
        #     loss, current = loss.item(), batch * len(X)
        #     print(f"loss: {loss:>7f}  [{current:>5d}/{size:>5d}]")

    TensorboardWriter(y, pred, epochs, loss, name)

epochs = 1000
for t in range(epochs):
    print(f"--------------Epoch {(t+1)/epochs * 100:.2f}%--------------")
    train(train_dataloader, model, loss_function, optimizer, t, "train")
    train(test_dataloader, model, loss_function, optimizer, t, "test")


torch.save(model.state_dict(), PATH + "SigleLayer.pth")
#モデルの保存
print("Done!")

--------------Epoch 0.10%--------------
--------------Epoch 0.20%--------------
--------------Epoch 0.30%--------------
--------------Epoch 0.40%--------------
--------------Epoch 0.50%--------------
--------------Epoch 0.60%--------------
--------------Epoch 0.70%--------------
--------------Epoch 0.80%--------------
--------------Epoch 0.90%--------------
--------------Epoch 1.00%--------------
--------------Epoch 1.10%--------------
--------------Epoch 1.20%--------------
--------------Epoch 1.30%--------------
--------------Epoch 1.40%--------------
--------------Epoch 1.50%--------------
--------------Epoch 1.60%--------------
--------------Epoch 1.70%--------------
--------------Epoch 1.80%--------------
--------------Epoch 1.90%--------------
--------------Epoch 2.00%--------------
--------------Epoch 2.10%--------------
--------------Epoch 2.20%--------------
--------------Epoch 2.30%--------------
--------------Epoch 2.40%--------------
--------------Epoch 2.50%--------------


In [ ]:
# 76
from torch.utils.data import TensorDataset, DataLoader
import torch
import torch.nn as nn
import torch.optim as optim

training_data = TensorDataset(X_train, Y_train)
train_dataloader = DataLoader(training_data,batch_size=128, shuffle=True)

optimizer = torch.optim.SGD(model.parameters(), lr=1e-3)

def train(dataloader, model, loss_fn, optimizer):
    size = len(dataloader.dataset)

    for batch, (X, y) in enumerate(dataloader):
        X, y = X.to(device), y.to(device)

        # 予測とlossの計算
        pred = model(X)
        loss = loss_fn(pred, y)

        # バックプロパゲーション
        optimizer.zero_grad()
        # モデルパラメータの勾配をリセットする
        loss.backward()
        # バックプロパゲーションを実行する
        optimizer.step()
        # パラメータの勾配を使用してパラメータの値を調整する

        # 10バッチごとに、現在のlossとトレーニングの進捗状況をprintして
        if batch % 10 == 0:
            loss, current = loss.item(), batch * len(X)
            print(f"loss: {loss:>7f}  [{current:>5d}/{size:>5d}]")

epochs = 10
for t in range(epochs):
    print(f"Epoch {t+1}\n-------------------------------")
    train(train_dataloader, model, loss_function, optimizer)
    torch.save(model.state_dict(), PATH + "checkpoint{}.pth".format(str(t).zfill(4)))
    print("-------------------------------\n")


print("Done!")

Epoch 1
-------------------------------
loss: 1.382760  [    0/10672]
loss: 1.383097  [ 1280/10672]
loss: 1.384498  [ 2560/10672]
loss: 1.382509  [ 3840/10672]
loss: 1.381811  [ 5120/10672]
loss: 1.383578  [ 6400/10672]
loss: 1.383421  [ 7680/10672]
loss: 1.383713  [ 8960/10672]
loss: 1.383289  [10240/10672]
-------------------------------

Epoch 2
-------------------------------
loss: 1.383326  [    0/10672]
loss: 1.383019  [ 1280/10672]
loss: 1.382980  [ 2560/10672]
loss: 1.383386  [ 3840/10672]
loss: 1.383588  [ 5120/10672]
loss: 1.382499  [ 6400/10672]
loss: 1.382619  [ 7680/10672]
loss: 1.383118  [ 8960/10672]
loss: 1.382896  [10240/10672]
-------------------------------

Epoch 3
-------------------------------
loss: 1.381657  [    0/10672]
loss: 1.382919  [ 1280/10672]
loss: 1.382648  [ 2560/10672]
loss: 1.382702  [ 3840/10672]
loss: 1.382275  [ 5120/10672]
loss: 1.383298  [ 6400/10672]
loss: 1.382985  [ 7680/10672]
loss: 1.382295  [ 8960/10672]
loss: 1.382681  [10240/10672]
----

In [ ]:
# 77
from torch.utils.data import TensorDataset, DataLoader
import torch
import torch.nn as nn
import torch.optim as optim
import time

training_data = TensorDataset(X_train, Y_train)
train_dataloader = DataLoader(training_data,batch_size=128, shuffle=True)

optimizer = torch.optim.SGD(model.parameters(), lr=1e-3)

bs_list = [2**i for i in range(20)]

def train(dataloader, model, loss_fn, optimizer):
    size = len(dataloader.dataset)

    for batch, (X, y) in enumerate(dataloader):
        X, y = X, y

        # 予測とlossの計算
        pred = model(X)
        loss = loss_fn(pred, y)

        # バックプロパゲーション
        optimizer.zero_grad()
        # モデルパラメータの勾配をリセットする
        loss.backward()
        # バックプロパゲーションを実行する
        optimizer.step()
        # パラメータの勾配を使用してパラメータの値を調整する

        # 10バッチごとに、現在のlossとトレーニングの進捗状況をprintして
        # if batch % 10 == 0:
        #     loss, current = loss.item(), batch * len(X)
        #     print(f"loss: {loss:>7f}  [{current:>5d}/{size:>5d}]")

epochs = 10
for bs in bs_list:
  time_1 = time.time()
  # 始めた時間
  for t in range(epochs):
    # print(f"Epoch {t+1}\n-------------------------------")
    train(train_dataloader, model, loss_function, optimizer)
    # torch.save(model.state_dict(), PATH + "checkpoint{}.pth".format(str(t).zfill(4)))


  time_2 = time.time()
  # 終わった時間
  running_time = (time_2 - time_1)/epochs
  print("Batch Size:",bs)
  print("time", running_time)
  print("-------------------------------\n")

print("Done!")

Batch Size: 1
time 0.14232702255249025
-------------------------------

Batch Size: 2
time 0.1463470935821533
-------------------------------

Batch Size: 4
time 0.138801908493042
-------------------------------

Batch Size: 8
time 0.1382387638092041
-------------------------------

Batch Size: 16
time 0.14165287017822265
-------------------------------

Batch Size: 32
time 0.13929524421691894
-------------------------------

Batch Size: 64
time 0.1421740770339966
-------------------------------

Batch Size: 128
time 0.1437765121459961
-------------------------------

Batch Size: 256
time 0.14362986087799073
-------------------------------

Batch Size: 512
time 0.1421347141265869
-------------------------------

Batch Size: 1024
time 0.14272735118865967
-------------------------------

Batch Size: 2048
time 0.14123976230621338
-------------------------------

Batch Size: 4096
time 0.14393141269683837
-------------------------------

Batch Size: 8192
time 0.14189279079437256
-----------

In [ ]:
# 78
from torch.utils.data import TensorDataset, DataLoader
import torch
import torch.nn as nn
import torch.optim as optim
import time

model = model.to(device)
if torch.cuda.device_count() > 1:
    model = nn.DataParallel(model)

training_data = TensorDataset(X_train, Y_train)
train_dataloader = DataLoader(training_data,batch_size=128, shuffle=True)

optimizer = torch.optim.SGD(model.parameters(), lr=1e-3)

bs_list = [2**i for i in range(20)]

def train(dataloader, model, loss_fn, optimizer):
    size = len(dataloader.dataset)

    for batch, (X, y) in enumerate(dataloader):
        X, y = X.to(device), y.to(device)
        # GPUで訓練する

        # 予測とlossの計算
        pred = model(X)
        loss = loss_fn(pred, y)

        # バックプロパゲーション
        optimizer.zero_grad()
        # モデルパラメータの勾配をリセットする
        loss.backward()
        # バックプロパゲーションを実行する
        optimizer.step()
        # パラメータの勾配を使用してパラメータの値を調整する

        # 10バッチごとに、現在のlossとトレーニングの進捗状況をprintして
        # if batch % 10 == 0:
        #     loss, current = loss.item(), batch * len(X)
        #     print(f"loss: {loss:>7f}  [{current:>5d}/{size:>5d}]")

epochs = 10
for bs in bs_list:
  time_1 = time.time()
  # 始めた時間
  for t in range(epochs):
    # print(f"Epoch {t+1}\n-------------------------------")
    train(train_dataloader, model, loss_function, optimizer)
    # torch.save(model.state_dict(), PATH + "checkpoint{}.pth".format(str(t).zfill(4)))


  time_2 = time.time()
  # 終わった時間
  running_time = (time_2 - time_1)/epochs
  print("Batch Size:",bs)
  print("time", running_time)
  print("-------------------------------\n")

print("Done!")

Batch Size: 1
time 0.14605965614318847
-------------------------------

Batch Size: 2
time 0.14837687015533446
-------------------------------

Batch Size: 4
time 0.15187735557556153
-------------------------------

Batch Size: 8
time 0.14531354904174804
-------------------------------

Batch Size: 16
time 0.14191982746124268
-------------------------------

Batch Size: 32
time 0.14415593147277833
-------------------------------

Batch Size: 64
time 0.1427739143371582
-------------------------------

Batch Size: 128
time 0.1441875457763672
-------------------------------

Batch Size: 256
time 0.1495656728744507
-------------------------------

Batch Size: 512
time 0.14712488651275635
-------------------------------

Batch Size: 1024
time 0.15443177223205568
-------------------------------

Batch Size: 2048
time 0.1527250051498413
-------------------------------

Batch Size: 4096
time 0.14840075969696045
-------------------------------

Batch Size: 8192
time 0.14659409523010253
--------

In [ ]:
# 79
class NeuralNetwork(nn.Module):
    def __init__(self, input_feature, output):
      super(NeuralNetwork, self).__init__()

      self.flatten = nn.Flatten()
      # 1次元配列に変換する

      self.linear_relu_stack = nn.Sequential(
        nn.Linear(input_feature, 300, bias=True),
        nn.ReLU(),
        nn.Linear(300, output, bias=True),
        nn.ReLU(),
        # このレイヤーはinput_featureからoutputへ線形変換をほどこする
        # 300—>4

        nn.Softmax(dim=1)
        # Softmax関数をほどこする
      )



    def forward(self, x):
      x = self.flatten(x)
      x = self.linear_relu_stack(x)

      return x

In [ ]:
# 79
from torch.utils.data import TensorDataset, DataLoader
import torch
import torch.nn as nn
import torch.optim as optim
import time

training_data = TensorDataset(X_train, Y_train)
train_dataloader = DataLoader(training_data,batch_size=128, shuffle=True)

optimizer = torch.optim.SGD(model.parameters(), lr=1e-3)

bs_list = [2**i for i in range(20)]

def train(dataloader, model, loss_fn, optimizer, name):
    size = len(dataloader.dataset)

    model.eval()

    test_loss, correct = 0, 0

    with torch.no_grad():
      for batch, (X, y) in enumerate(dataloader):
        X, y = X.to(device), y.to(device)
        # GPUで訓練する

        # 予測とlossの計算
        pred = model(X)
        loss = loss_fn(pred, y)

        test_loss += loss.item()
        # lossを合算する
        correct += (pred.argmax(1) == y).type(torch.float).sum().item()
        # correctを合算する

    # 平均lossを計算する
    test_loss /= size
    # 平均correctを計算する
    correct /= size
    print(f"{name} Error: \n Accuracy: {(100*correct):>0.1f}%, Avg loss: {test_loss:>8f} \n")

epochs = 10
for t in range(epochs):
  # print(f"Epoch {t+1}\n-------------------------------")
  train(train_dataloader, model, loss_function, optimizer, "train")
  train(test_dataloader, model, loss_function, optimizer, "test")



print("Done!")

train Error: 
 Accuracy: 77.8%, Avg loss: 0.007936 

test Error: 
 Accuracy: 79.8%, Avg loss: 0.008166 

train Error: 
 Accuracy: 77.8%, Avg loss: 0.007939 

test Error: 
 Accuracy: 79.8%, Avg loss: 0.008195 

train Error: 
 Accuracy: 77.8%, Avg loss: 0.007932 

test Error: 
 Accuracy: 79.8%, Avg loss: 0.008164 

train Error: 
 Accuracy: 77.8%, Avg loss: 0.007937 

test Error: 
 Accuracy: 79.8%, Avg loss: 0.008152 

train Error: 
 Accuracy: 77.8%, Avg loss: 0.007939 

test Error: 
 Accuracy: 79.8%, Avg loss: 0.008183 

train Error: 
 Accuracy: 77.8%, Avg loss: 0.007932 

test Error: 
 Accuracy: 79.8%, Avg loss: 0.008150 

train Error: 
 Accuracy: 77.8%, Avg loss: 0.007936 

test Error: 
 Accuracy: 79.8%, Avg loss: 0.008189 

train Error: 
 Accuracy: 77.8%, Avg loss: 0.007936 

test Error: 
 Accuracy: 79.8%, Avg loss: 0.008188 

train Error: 
 Accuracy: 77.8%, Avg loss: 0.007934 

test Error: 
 Accuracy: 79.8%, Avg loss: 0.008189 

train Error: 
 Accuracy: 77.8%, Avg loss: 0.007934 

te